Introduction


**Table of contents**<a id='toc0_'></a>    
- 1. [Data preparation](#toc1_)    
  - 1.1. [Tokenize sentence and aspect BIO encoding class](#toc1_1_)    
  - 1.2. [Load data](#toc1_2_)    
  - 1.3. [Inspect tagging issues](#toc1_3_)    
  - 1.4. [Merge conflict to negative & remove neutral](#toc1_4_)    
  - 1.5. [Convert df to HuggingFace datasets](#toc1_5_)    
- 2. [EDA](#toc2_)    
- 3. [Random forest](#toc3_)    
- 4. [Logistic Regression](#toc4_)    
- 5. [CRF](#toc5_)    
- 6. [Bi-LSTM](#toc6_)    
- 7. [BERT](#toc7_)    
  - 7.1. [Define model](#toc7_1_)    
    - 7.1.1. [Define tokenizer](#toc7_1_1_)    
    - 7.1.2. [Define token classification model](#toc7_1_2_)    
  - 7.2. [Data preparation](#toc7_2_)    
  - 7.3. [Performance metrics](#toc7_3_)    
  - 7.4. [Training](#toc7_4_)    
    - 7.4.1. [Define trainer](#toc7_4_1_)    
    - 7.4.2. [Training results](#toc7_4_2_)    
  - 7.5. [Model performance](#toc7_5_)    
    - 7.5.1. [Perform prediction on validation dataset](#toc7_5_1_)    
    - 7.5.2. [Model performance](#toc7_5_2_)    
  - 7.6. [Error analysis](#toc7_6_)    
    - 7.6.1. [Group by word token](#toc7_6_1_)    
    - 7.6.2. [Group by Tag ID](#toc7_6_2_)    
  - 7.7. [Save model](#toc7_7_)    
  - 7.8. [Load saved model](#toc7_8_)    
    - 7.8.1. [Load model manually](#toc7_8_1_)    
    - 7.8.2. [Pipeline](#toc7_8_2_)    
- 8. [DistilBert with resampling](#toc8_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import os 
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0' # this setting is needed to run NN on my Mac

import re
import pandas as pd
import numpy as np
from collections import Counter
from collections import defaultdict

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from highlight_text import HighlightText, ax_text, fig_text


from nltk.tokenize import TreebankWordTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from seqeval.metrics import f1_score, classification_report

# pip install torch==2.2.0 torchtext --index-url https://download.pytorch.org/whl/test/cpu
# pip install torch==2.3.0.dev20240121 # this does not work

# pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu # run this in CLI before running the notebook

# import torch
# if torch.backends.mps.is_available():
#     mps_device = torch.device("mps")
#     x = torch.ones(1, device=mps_device)
#     print (x)
# else:
#     print ("MPS device not found.")

# 1. <a id='toc1_'></a>[Data preparation](#toc0_)

## 1.1. <a id='toc1_1_'></a>[Tokenize sentence and aspect BIO encoding class](#toc0_)

In [10]:
class SentenceToken:
  '''
    SentenceToken

    This class takes care of word tokenize and tagging aspect entities
  '''
  def __init__(self, sentence, aspect_type=None, aspects=None, sentence_id=None):
    
    if sentence_id is not None:
      print(sentence_id)

    self.sentence_id = sentence_id
    self.sentence = sentence.replace(u"\u00A0", " ").replace(u'\xa0',' ') # replace unicode space character
                            
    self.aspect_bio_tags = None
    self.unified_aspect_bio_tags = None
    self.token_span = None
    self.space_pre_token = None

    # Tokenize sentence
    self.__tokenize_sentence(self.sentence)

    if aspect_type == 'dict':
      self.set_aspect_tagging_from_dict(aspects)
    elif aspect_type == 'bio':
      self.set_aspect_bio_tags(aspects)
    elif aspect_type == 'unified bio':
      self.set_aspect_unified_bio_tags(aspects)
  
  def __tokenize_sentence(self, sentence):
    '''
    __tokenize_sentence

    Break sentence into word token span
    '''
    token_span = list(TreebankWordTokenizer().span_tokenize(sentence))
    
    self.token_span = token_span
    self.space_pre_token = [True if sentence[k[0]-1:k[0]] == ' ' else False for i,k in enumerate(token_span)]

  def set_aspect_tagging_from_dict(self, aspects):
    '''
    set_aspect_tagging_from_dict

    Calculate & assign aspect entities to token given an array of aspects (term, start_index, to_index, and polarity)
    '''
    polarity_map = {'positive':'POS'
              ,'negative': 'NEG'
              ,'conflict': 'CON'
              ,'neutral': 'NEU'}
    
    bio_tags = ['O'] * len(self.token_span)
    unified_bio_tags = bio_tags

    for x in aspects:
      if x['term'] != '':
        aspect_from = int(x['from'])
        aspect_to = int(x['to'])
        polarity = '-' + polarity_map[x['polarity']]

        aspect_from_index = [i for i, v in enumerate(self.token_span) if (v[0] <= aspect_from) & (v[1] >= aspect_from)][0]
        aspect_to_index = [i for i, v in enumerate(self.token_span) if (v[0] <= aspect_to) & (v[1] >= aspect_to)][0]
      
        aspect_length = aspect_to_index - aspect_from_index
        bio_tags = bio_tags[:aspect_from_index] + ['B'] + ['I'] * (aspect_length) + bio_tags[aspect_to_index+1:]
        unified_bio_tags = unified_bio_tags[:aspect_from_index] + ['B' + polarity] + ['I'+ polarity] * (aspect_length) + unified_bio_tags[aspect_to_index+1:]

    self.set_aspect_bio_tags(bio_tags)
    self.set_aspect_unified_bio_tags(unified_bio_tags)

  def rebuild_sentence_from_token(self):
    '''
    rebuild_sentence_from_token

    Return sentence built from computed tokens
    '''
    return ''.join([(' ' if self.space_pre_token[i] else '') + self.sentence[k[0]:k[1]] for i, k in enumerate(self.token_span)])
 
  def set_aspect_bio_tags(self, aspect_bio_tags):
    '''
    set_aspect_bio_tags

    Setter method to set aspect_unified_bio_tags and aspect_bio_tags
    '''
    self.aspect_bio_tags = aspect_bio_tags
    self.aspect_unified_bio_tags = aspect_bio_tags

  def set_aspect_unified_bio_tags(self, aspect_unified_bio_tags):
    ''''
    set_aspect_unified_bio_tags
    
    Setter method to set aspect_unified_bio_tags and aspect_bio_tags
    '''
    self.aspect_unified_bio_tags = aspect_unified_bio_tags
    self.aspect_bio_tags = [k[0:1] for k in aspect_unified_bio_tags]

  def get_tokens(self):
    '''
    get_tokens()
    Return an array of sentence word tokens
    '''
    return [self.sentence[k[0]:k[1]] for k in self.token_span]
  
  def check_rebuild_sentence_from_token(self):
    '''
    check_rebuild_sentence_from_token()

    This is a test / debugger function.
    This help validating if we have computed the sentence to token properly and whether we can re-compute the exact sentence from information stored.
    '''
    return re.sub(r'\s+', ' ',self.sentence.strip()) == self.rebuild_sentence_from_token().strip()
  
  def check_rebuild_aspect_terms(self, aspect_dict):
    '''
    check_rebuild_aspect_terms(aspect_dict)

    This is a test / debugger fucntion. 
    This help validate if we have compute the correct aspect terms as given by the aspect dict

    INPUT:
    aspect dict: array of aspect dictionaries in the following format
      [{'term': 'storage', 
       'polarity': 'positive', 
       'from': '14', 
       'to': '21'}]
    '''
    aspect_dict = sorted(aspect_dict, key=lambda d: int(d['from']))
    aspect_input = [k['term'].replace(u"\u00A0", " ").replace(u'\xa0',' ') for k in aspect_dict if k['term'] != '' ]
    aspect_computed = []
    aspect = ''
    
    for i,k in enumerate(self.aspect_bio_tags):
      token = self.sentence[self.token_span[i][0]:self.token_span[i][1]]
      
      if k == 'B':
        if (self.aspect_bio_tags[i-1] == 'B' if i > 0 else False):
          aspect_computed.append(aspect)
        aspect = token
      elif k == 'I':
        aspect += ' ' * ((self.token_span[i][0] -  self.token_span[i-1][1]) if i > 0 else 0) + token
      
      if (aspect != '') & ((k == 'O') or (i == (len(self.aspect_bio_tags) - 1))):
          aspect_computed.append(aspect)
          aspect = ''

    return [aspect_input == aspect_computed, aspect_input, aspect_computed]

  def __str__(self):
    return self.rebuild_sentence_from_token()


## 1.2. <a id='toc1_2_'></a>[Load data](#toc0_)

In [11]:
def reload_Data():
  df_train = pd.read_json('data/laptop/train.json')
  # First, I will need to drop some duplicated data in our training dataset, as identified in the EDA process.
  # We have removed 12 duplicated records in our training dataset
  df_train.drop_duplicates(subset='text', inplace=True)
  print('df_train shape: ', df_train.shape)

  df_val = pd.read_json('data/laptop/validate.json') # This will only be used for the very last step to evaluate how well the model is, but is input now for validating the BIO tagging to ensure the function works properly
  print('df_val shape: ', df_val.shape)

  df_train['sentence_token'] = df_train.apply(lambda x: SentenceToken(x['text'], 'dict', x['aspects']), axis=1)
  df_train['sentence_check'] = df_train.apply(lambda x: x['sentence_token'].check_rebuild_sentence_from_token(), axis=1)
  df_train['aspect_check'] = df_train.apply(lambda x: x['sentence_token'].check_rebuild_aspect_terms(x['aspects']), axis=1)
  df_train['aspect_check_TF'] = df_train.apply(lambda x: x['sentence_token'].check_rebuild_aspect_terms(x['aspects'])[0], axis=1)
  df_train['tokens'] = df_train.apply(lambda x: x['sentence_token'].get_tokens(), axis=1)
  df_train['tags'] = df_train.apply(lambda x: x['sentence_token'].aspect_unified_bio_tags, axis=1)

  df_val['sentence_token'] = df_val.apply(lambda x: SentenceToken(x['text'], 'dict', x['aspects']), axis=1)
  df_val['sentence_check'] = df_val.apply(lambda x: x['sentence_token'].check_rebuild_sentence_from_token(), axis=1)
  df_val['aspect_check'] = df_val.apply(lambda x: x['sentence_token'].check_rebuild_aspect_terms(x['aspects']), axis=1)
  df_val['aspect_check_TF'] = df_val.apply(lambda x: x['sentence_token'].check_rebuild_aspect_terms(x['aspects'])[0], axis=1)
  df_val['tokens'] = df_val.apply(lambda x: x['sentence_token'].get_tokens(), axis=1)
  df_val['tags'] = df_val.apply(lambda x: x['sentence_token'].aspect_unified_bio_tags, axis=1)
  return df_train, df_val

In [12]:
df_train, df_test = reload_Data()
print('# of df_train records having tokenizing issues: ', len(df_train[df_train['sentence_check']==False]))
print('# of df_train records having aspect bio tagging issues: ', len(df_train[df_train['aspect_check_TF']==False]))
print('# of df_test records having tokenizing issues: ', len(df_test[df_test['sentence_check']==False]))
print('# of df_test records having aspect bio tagging issues: ', len(df_test[df_test['aspect_check_TF']==False]))

df_train shape:  (3036, 3)
df_val shape:  (800, 3)
# of df_train records having tokenizing issues:  0
# of df_train records having aspect bio tagging issues:  36
# of df_test records having tokenizing issues:  0
# of df_test records having aspect bio tagging issues:  9


## 1.3. <a id='toc1_3_'></a>[Inspect tagging issues](#toc0_)

In [13]:
print(df_train[df_train['aspect_check_TF']==False].index)

Index([ 125,  140,  220,  293,  374,  375,  431,  612,  656,  834,  922,  924,
        953,  999, 1031, 1374, 1456, 1502, 1631, 1716, 1936, 1958, 2113, 2160,
       2244, 2392, 2502, 2533, 2587, 2606, 2783, 2831, 2842, 2876, 2930, 2940],
      dtype='int64')


In [14]:
num = 2606

print(df_train.loc[num]['aspects'])
print(df_train.loc[num]['text'])
print(df_train.loc[num]['aspect_check'])
print(df_train.loc[num]['tokens'])

[{'term': 'delivery service', 'polarity': 'negative', 'from': '59', 'to': '75'}]
After way too many times sending the thing in for repairs (delivery service was slow, and without the laptop I had no access to the internet, and thus no way of tracking it to find out when I might hope to see my computer again), it finally kicked the bucket after just over 2 years.
[False, ['delivery service'], ['(delivery service']]
['After', 'way', 'too', 'many', 'times', 'sending', 'the', 'thing', 'in', 'for', 'repairs', '(', 'delivery', 'service', 'was', 'slow', ',', 'and', 'without', 'the', 'laptop', 'I', 'had', 'no', 'access', 'to', 'the', 'internet', ',', 'and', 'thus', 'no', 'way', 'of', 'tracking', 'it', 'to', 'find', 'out', 'when', 'I', 'might', 'hope', 'to', 'see', 'my', 'computer', 'again', ')', ',', 'it', 'finally', 'kicked', 'the', 'bucket', 'after', 'just', 'over', '2', 'years', '.']


Most of the tagging issues due to word that are not separated properly from special characters/ punctuations. The issue is unavoidable in practice as reviews may not adherent to perfect grammar.

I have tried to fix this issues to have 100% accuracy with further token breakdown to match the specified aspect tokens, however, this can break some of the standard logics for word tokenizer and further modelling. 

Therefore, I decided to include a whole token where the aspect may start or end, even if the index is in the middle of token, which may results with aspect tokens that could include extra characters than planned. This is the risk we will accept for this approach, and we can perform a cleaning process to remove these extra characters during implementation with actual use cases.

## 1.4. <a id='toc1_4_'></a>[Merge conflict to negative & remove neutral](#toc0_)

In [15]:
df_train['tags'] = df_train['tags'].apply(lambda x: [tag[:2] + 'NEG' if tag[2:] == 'CON' else tag for tag in x ])
df_test['tags'] = df_test['tags'].apply(lambda x: [tag[:2] + 'NEG' if tag[2:] == 'CON' else tag for tag in x])

In [16]:
# df_train['tags'] = df_train['tags'].apply(lambda x: ['O' if tag[2:] == 'NEU' else tag for tag in x])
# df_test['tags'] = df_test['tags'].apply(lambda x: ['O' if tag[2:] == 'NEU' else tag for tag in x])

## 1.5. <a id='toc1_5_'></a>[Convert df to HuggingFace datasets](#toc0_)

In [17]:
import datasets
from datasets import Dataset, DatasetDict, Features, Sequence, Value, ClassLabel

Convert aspect tag to id

In [18]:
# Generate list of unique aspect tags
tags = list(set(sum(df_train['tags'],[])))
tags.sort()
tag2idx = {k:i for i,k in enumerate(tags)}

# Convert aspect tag text to ids
df_train['tags_idx'] = df_train['tags'].apply(lambda x: [tag2idx[k] for k in x])
df_test['tags_idx'] = df_test['tags'].apply(lambda x: [tag2idx[k] for k in x])

In [19]:
tags

['B-NEG', 'B-NEU', 'B-POS', 'I-NEG', 'I-NEU', 'I-POS', 'O']

Convert pandas to HuggingFace datasets

In [20]:
# Split train into train & validation set
df_train_ori, df_val = train_test_split(df_train, test_size=0.3, random_state=42,)

In [22]:
# Define dataset features
features = Features({'tokens': Sequence(Value(dtype='string', id=None)),
                    'tags_idx': Sequence(ClassLabel(names=tags))
                    })

tds = Dataset.from_pandas(df_train_ori[['tokens','tags_idx']], features=features, preserve_index=False)
vds = Dataset.from_pandas(df_val[['tokens','tags_idx']], features=features,  preserve_index=False)
tsds = Dataset.from_pandas(df_test[['tokens','tags_idx']], features=features, preserve_index=False)

ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds
ds['test'] = tsds

print(ds)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags_idx'],
        num_rows: 2125
    })
    validation: Dataset({
        features: ['tokens', 'tags_idx'],
        num_rows: 911
    })
    test: Dataset({
        features: ['tokens', 'tags_idx'],
        num_rows: 800
    })
})


Check for total counts per aspect type in each data split

In [23]:
# Convert aspect idx to aspect aspect text
def create_tag_names(batch):
    return {"tags": [ds["train"].features["tags_idx"].feature.int2str(idx) for idx in batch["tags_idx"]]}

ds = ds.map(create_tag_names)

Map:   0%|          | 0/2125 [00:00<?, ? examples/s]

Map:   0%|          | 0/911 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [24]:
# Perform aspect type counts per each dataset split
split2freqs = defaultdict(Counter)
for split, dataset in ds.items():
    for row in dataset["tags"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
                
pd.DataFrame.from_dict(split2freqs, orient="index")

,NEG,POS,NEU
train,582,675,304
validation,325,312,157
test,144,340,169


# 2. <a id='toc2_'></a>[EDA](#toc0_)

In [ ]:
# 

# Model performance class

In [42]:
class ModelPerformanceLog:
  
  def __init__(self, y_true):
    self.y_true = y_true
    self.log = {}

  def add_model_performance(self, model_name, y_pred, model=None): # Model can be path to the model or something like that
    f1, clf_report = self.__calculate_metrics(y_pred)

    model_perf = {'y_pred': y_pred
                  ,'model': model
                  ,'f1': f1.round(2)
                  ,'report': clf_report
                  }
    
    self.log[model_name] = model_perf

  def __calculate_metrics(self, y_pred):
    return f1_score(self.y_true, y_pred), classification_report(self.y_true, y_pred, zero_division=0)
  
  def get_logs(self):
    return pd.DataFrame(list(self.log.values()), index=self.log.keys())
  
    

In [43]:
modelPerformanceLog = ModelPerformanceLog(ds['validation']['tags_idx'])
# modelPerformanceLog.get_logs()

# 3. <a id='toc3_'></a>[Random forest](#toc0_)

## Prepare data

# 4. <a id='toc4_'></a>[Logistic Regression](#toc0_)

# 5. <a id='toc5_'></a>[CRF](#toc0_)

# 6. <a id='toc6_'></a>[Bi-LSTM](#toc0_)

# 7. <a id='toc7_'></a>[BERT](#toc0_)

In [ ]:

import torch
import torch.nn as nn
from torch.nn.functional import cross_entropy

from transformers import AutoConfig, DistilBertConfig, AutoTokenizer, TrainingArguments, DataCollatorForTokenClassification, Trainer
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.distilbert.modeling_distilbert import DistilBertModel
from transformers.models.distilbert.modeling_distilbert import DistilBertPreTrainedModel

device = torch.device('mps') # This is required for Mac

## 7.1. <a id='toc7_1_'></a>[Define model](#toc0_)

In [ ]:
distilbert_model_name = "distilbert-base-uncased"

### 7.1.1. <a id='toc7_1_1_'></a>[Define tokenizer](#toc0_)

In [ ]:
distilbert_tokenizer = AutoTokenizer.from_pretrained(distilbert_model_name)

### 7.1.2. <a id='toc7_1_2_'></a>[Define token classification model](#toc0_)

In [ ]:
tags = ds['train'].features['tags_idx'].feature

index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
tags

In [ ]:
distilbert_config = AutoConfig.from_pretrained(distilbert_model_name, 
                                         num_labels=tags.num_classes,
                                         id2label=index2tag, label2id=tag2index)

In [ ]:
# Define token weights for loss calculation

flatten_tags = sum(ds['train']['tags'],[])
token_weights = Counter(flatten_tags)
token_weights = [token_weights[x] for x in tags.names]
token_weights = torch.tensor(1 / np.array(token_weights), dtype=torch.float32).to(device)
token_weights

In [ ]:
class DistilBertForABSA(DistilBertPreTrainedModel):
    config_class = DistilBertConfig

    def __init__(self, config):
        super().__init__(config)
        # Roberta body
        self.num_labels = config.num_labels
        self.distilbert = DistilBertModel(config)#, add_pooling_layer=False)

        # Classification head
        self.dropout = nn.Dropout(config.dropout)#hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        # Load and initialize weights from pretrained
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, 
                labels=None, **kwargs):
        
        # Use model body to get encoder representations
        outputs = self.distilbert(input_ids, attention_mask=attention_mask,
                            #    token_type_ids=token_type_ids,
                                 **kwargs)
        
        # Apply classifier to encoder representation (model head)
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        
        # Calculate losses
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            
        # Return model output object
        return TokenClassifierOutput(loss=loss, logits=logits, 
                                     hidden_states=outputs.hidden_states, 
                                     attentions=outputs.attentions)
     

## 7.2. <a id='toc7_2_'></a>[Data preparation](#toc0_)

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = distilbert_tokenizer(examples["tokens"], truncation=True, 
                                      is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples["tags_idx"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
def encode_dataset(corpus):
    return corpus.map(tokenize_and_align_labels, batched=True, 
                      remove_columns=['tags_idx', 'tokens','tags']
                      )

In [ ]:
# hide_output
ds_encoded = encode_dataset(ds)

## 7.3. <a id='toc7_3_'></a>[Performance metrics](#toc0_)

In [ ]:
def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore label IDs = -100
            if label_ids[batch_idx, seq_idx] != -100:
                index2tag = distilbert_config.id2label.copy()
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

In [ ]:
# Define performance metrics
def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions, 
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred, average='macro')}

## 7.4. <a id='toc7_4_'></a>[Training](#toc0_)

### 7.4.1. <a id='toc7_4_1_'></a>[Define trainer](#toc0_)

In [ ]:
# Define training arguments
num_epochs = 15
batch_size = 24
logging_steps = len(ds["train"]) // batch_size
model_name = f"{distilbert_model_name}-absa-resample"

training_args = TrainingArguments(
    output_dir="model/" + model_name, log_level="error", num_train_epochs=num_epochs, 
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=batch_size, evaluation_strategy="epoch", 
    save_steps=1e6, weight_decay=0.01, disable_tqdm=False, 
    logging_steps=logging_steps, push_to_hub=False)

# Define data collator for data batching 
data_collator = DataCollatorForTokenClassification(distilbert_tokenizer)
     
# Model init
def model_init():
    return (DistilBertForABSA
            .from_pretrained(distilbert_model_name, config=distilbert_config)
            .to(device))
     
trainer = Trainer(model_init=model_init, args=training_args, 
                  data_collator=data_collator, compute_metrics=compute_metrics,
                  train_dataset=ds_encoded["train"],
                  eval_dataset=ds_encoded["validation"], 
                  tokenizer=distilbert_tokenizer)

In [ ]:
trainer.train()

### 7.4.2. <a id='toc7_4_2_'></a>[Training results](#toc0_)

In [ ]:
df = pd.DataFrame(trainer.state.log_history)[['epoch','loss' ,'eval_loss','eval_f1']]
df = df.rename(columns={"epoch":"Epoch","loss": "Training Loss", "eval_loss": "Validation Loss", 'eval_f1':'F1'})
df['Epoch'] = df["Epoch"].apply(lambda x: round(x))
df['Training Loss'] = df["Training Loss"].ffill()
df[['Validation Loss', 'F1']] = df[['Validation Loss', 'F1']].bfill().ffill()
df.drop_duplicates()

## 7.5. <a id='toc7_5_'></a>[Model performance](#toc0_)

### 7.5.1. <a id='toc7_5_1_'></a>[Perform prediction on validation dataset](#toc0_)

In [ ]:
def forward_pass_with_label(batch):
    # Convert dict of lists to list of dicts suitable for data collator
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    # Pad inputs and labels and put all tensors on device
    batch = data_collator(features)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    with torch.no_grad():
        # Pass data through model  
        output = trainer.model(input_ids, attention_mask)

        # Logit.size: [batch_size, sequence_length, classes]
        # Predict class with largest logit value on classes axis
        predicted_label = torch.argmax(output.logits, axis=-1).cpu().numpy()
    # Calculate loss per token after flattening batch dimension with view
    loss = cross_entropy(output.logits.view(-1, tags.num_classes), 
                         labels.view(-1), reduction="none")
    # Unflatten batch dimension and convert to numpy array
    loss = loss.view(len(input_ids), -1).cpu().numpy()

    return {"loss":loss, "predicted_label": predicted_label}
     

In [ ]:
validation_set = ds_encoded["validation"]
validation_set = validation_set.map(forward_pass_with_label, batched=True, batch_size=32)
df_validation = validation_set.to_pandas()

In [ ]:
# Cleanup & conver id2text 
index2tag = trainer.model.config.id2label.copy()
index2tag[-100] = "IGN"
df_validation["input_tokens"] = df_validation["input_ids"].apply(
    lambda x: distilbert_tokenizer.convert_ids_to_tokens(x))
df_validation["predicted_label"] = df_validation["predicted_label"].apply(
    lambda x: [index2tag[i] for i in x])
df_validation["labels"] = df_validation["labels"].apply(
    lambda x: [index2tag[i] for i in x])
df_validation['loss'] = df_validation.apply(
    lambda x: x['loss'][:len(x['input_ids'])], axis=1) # Remove padding tokens
df_validation['predicted_label'] = df_validation.apply(
    lambda x: x['predicted_label'][:len(x['input_ids'])], axis=1) # Remove padding tokens

# Flatten the outputs
df_tokens = df_validation.apply(pd.Series.explode)
df_tokens = df_tokens.query("labels != 'IGN'")
df_tokens["loss"] = df_tokens["loss"].astype(float).round(2)

### 7.5.2. <a id='toc7_5_2_'></a>[Model performance](#toc0_)

In [ ]:
df_validation_metrics = df_validation.copy()

df_validation_metrics['predicted_label'] = df_validation_metrics.apply(lambda x: [x['predicted_label'][i] for i,k in enumerate(x['labels']) if k != 'IGN' ], axis=1)
df_validation_metrics['labels'] = df_validation_metrics.apply(lambda x: [k for i,k in enumerate(x['labels']) if k != 'IGN' ], axis=1)

print(f"F1 Score: {f1_score(df_validation_metrics['labels'], df_validation_metrics['predicted_label']).round(2)}")
print(classification_report(df_validation_metrics['labels'], df_validation_metrics['predicted_label'], zero_division=0))

## 7.6. <a id='toc7_6_'></a>[Error analysis](#toc0_)

### 7.6.1. <a id='toc7_6_1_'></a>[Group by word token](#toc0_)

In [ ]:
(
    df_tokens.groupby("input_tokens")[["loss"]]
    .agg(["count", "mean", "sum"])
    .droplevel(level=0, axis=1)  # Get rid of multi-level columns
    .sort_values(by="sum", ascending=False)
    .reset_index()
    .round(2)
    .head(10)
    .T
)

In [ ]:
(
  df_tokens[df_tokens['input_tokens']=='battery'].groupby(["predicted_label", 'labels'])[["loss"]]
    .agg(["count", "mean", "sum"])
    .droplevel(level=0, axis=1)  # Get rid of multi-level columns
    .sort_values(by="sum", ascending=False)
    .reset_index()
    .round(2)
    .head(10)
    .T
)

### 7.6.2. <a id='toc7_6_2_'></a>[Group by Tag ID](#toc0_)

In [ ]:
(
    df_tokens.groupby("labels")[["loss"]] 
    .agg(["count", "mean", "sum"])
    .droplevel(level=0, axis=1)
    .sort_values(by="mean", ascending=False)
    .reset_index()
    .round(2)
    .T
)

In [ ]:
def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()

In [ ]:
plot_confusion_matrix(df_tokens["labels"], df_tokens["predicted_label"],
                      list(tag2index.keys()))


In [ ]:
def get_samples(df):
    for _, row in df.iterrows():
        labels, preds, tokens, losses = [], [], [], []
        for i, mask in enumerate(row["attention_mask"]):
            if i not in {0, len(row["attention_mask"])}:
                labels.append(row["labels"][i])
                preds.append(row["predicted_label"][i])
                tokens.append(row["input_tokens"][i])
                losses.append(f"{row['loss'][i]:.2f}")
        df_tmp = pd.DataFrame({"tokens": tokens, "labels": labels, 
                               "preds": preds, "losses": losses}).T
        yield df_tmp

df_validation["total_loss"] = df_validation["loss"].apply(sum)
df_tmp = df_validation.sort_values(by="total_loss", ascending=False).head(3)

for sample in get_samples(df_tmp):
    display(sample.T)
     

Errors could be from human / annotation errors: United Nations is ORG, not PER, similar to Central African Republic. This can happen as data was annotated using rule based, it is better with human annotations, but mistakes can always occur.

In [ ]:
df_tmp = df_validation.loc[df_validation["input_tokens"].apply(lambda x: u"\u2581(" in x)].head(2)
for sample in get_samples(df_tmp):
    display(sample.T)

## 7.7. <a id='toc7_7_'></a>[Save model](#toc0_)

In [ ]:
trainer.save_model('model/distilbert-base-uncased-absa-downsample-1-3')


## 7.8. <a id='toc7_8_'></a>[Load saved model](#toc0_)

### 7.8.1. <a id='toc7_8_1_'></a>[Load model manually](#toc0_)

In [ ]:
from transformers import AutoModelForTokenClassification

# Reload the model
loaded_model = AutoModelForTokenClassification.from_pretrained('model/distilbert-base-uncased-absa-downsample-1-3').to(device) #output_model_dir
loaded_tokenizer = AutoTokenizer.from_pretrained('model/distilbert-base-uncased-absa-downsample-1-3')


In [ ]:
num = 99
sample_input = df_train.iloc[num]['text']
print(sample_input)
print(df_train.iloc[num]['aspects'])

In [ ]:
# Assuming you have some input data
input_data = [sample_input]

# Tokenize and get predictions
inputs = loaded_tokenizer(input_data, is_split_into_words=True, return_tensors="pt")

input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

# Make predictions
with torch.no_grad():
  outputs = loaded_model(input_ids, attention_mask)

predicted_label_idx = torch.argmax(outputs.logits, axis=-1).cpu().numpy()
df_res = pd.DataFrame({'predicted_label': predicted_label_idx.tolist(), 
                      'input_ids': inputs['input_ids'].numpy().tolist()}
                      )

In [ ]:
index2tag_new = loaded_model.config.id2label.copy()
index2tag_new[-100] = "IGN"
df_res["input_tokens"] = df_res["input_ids"].apply(
    lambda x: loaded_tokenizer.convert_ids_to_tokens(x))
df_res["predicted_label_text"] = df_res["predicted_label"].apply(
    lambda x: [index2tag_new[i] for i in x])
df_res['predicted_label'] = df_res.apply(
    lambda x: x['predicted_label'][:len(x['input_ids'])], axis=1)
df_res['predicted_label_text'] = df_res.apply(
    lambda x: x['predicted_label_text'][:len(x['input_ids'])], axis=1)

df_res_tokens = df_res.apply(pd.Series.explode)

df_res_tokens

### 7.8.2. <a id='toc7_8_2_'></a>[Pipeline](#toc0_)

In [ ]:
from transformers import pipeline

token_classifier = pipeline(
    "token-classification", model='model/distilbert-base-uncased-absa-downsample-1-3', aggregation_strategy="simple"
)
token_classifier(sample_input)

# 8. <a id='toc8_'></a>[DistilBert with resampling](#toc0_)

In [ ]:
# def custom_sampling_based_on_score(df, score_column='Score', target_samples_per_score=100):
#     sampled_data = []
    
#     # Calculate the sampling probabilities based on the scores
#     scores = df[score_column].unique() 
#     probabilities = scores / sum(scores)

#     probabilities_df = pd.DataFrame({'probability': probabilities}, index=scores)
#     probabilities_df['num_samples'] = np.round(probabilities_df['probability'] * target_samples_per_score).astype(int)
#     probabilities_df.loc[probabilities_df.index[-1], 'num_samples'] += target_samples_per_score - probabilities_df['num_samples'].sum()
#     # Determine the number of samples to draw for each score
#     # samples_per_score = np.round(probabilities * target_samples_per_score).astype(int)
    
#     # Ensure the total number of samples is equal to the target_samples_per_score
#     # samples_per_score[-1] += target_samples_per_score - samples_per_score.sum()
    
#     # Iterate over unique scores and sample sentences accordingly
#     for score_value in probabilities_df.index:
#         num_samples = probabilities_df.loc[score_value]['num_samples']
        
#         # Extract sentences with a specific score
#         subset = df[df[score_column] == score_value]
        
#         # Sample sentences based on the calculated number of samples
#         sampled_subset = subset.sample(n=int(num_samples), replace=True, random_state=42)
        
#         # Append the sampled subset to the list
#         sampled_data.append(sampled_subset)
    
#     # Concatenate the sampled subsets to create the final upsampled DataFrame
#     custom_sampled_df = pd.concat(sampled_data, ignore_index=True)
    
#     return custom_sampled_df



In [ ]:
# def custom_sampling_based_on_score(df):
#     sampled_data = []
    
#     df['score'] = df['tags'].apply(lambda x: sum([ 0 if tag.startswith('O') else 1 for tag in x  ]) + 1)
    
#     custom_sampled_df = pd.DataFrame(columns=df.columns)

#     for idx in range(len(df)):
#         num_samples = df.iloc[idx]['score']
#         sent = df.iloc[idx].to_dict()

#         for i in range(num_samples):
#         # Append the sampled subset to the list
#           sampled_data.append(sent)
#           # custom_sampled_df = pd.concat([custom_sampled_df, df.iloc[idx]], ignore_index=True, axis = 0)
#     # Concatenate the sampled subsets to create the final upsampled DataFrame
#     custom_sampled_df = pd.concat([custom_sampled_df, pd.DataFrame(sampled_data)], ignore_index=True)
    
#     return custom_sampled_df



In [ ]:

# Set the target number of samples per score
target_samples_per_score = 3000

# Use the custom sampling method
custom_sampled_df = custom_sampling_based_on_score(df_train_ori.copy(), 'nsCRD')

# Check the distribution of scores in the custom sampled DataFrame
# print(custom_sampled_df['entities_score'].value_counts())
print(len(custom_sampled_df))

In [ ]:
# Perform aspect type counts per each dataset split
split2freqs = defaultdict(Counter)

for row in df_train_ori["tags"]:
    for tag in row:
        # if tag.startswith("B"):
        #     tag_type = tag.split("-")[1]
        split2freqs['train'][tag] += 1
                
pd.DataFrame.from_dict(split2freqs, orient="index")

In [ ]:
df_train_ori['score'] = df_train_ori['tags'].apply(lambda x:sum([ 1 for tag in x if tag != 'O']))

In [ ]:
print(len(df_train_ori))
print(len(df_train_ori[df_train_ori['score']==0]))

In [ ]:
from random import randint
df_train_ori['select'] = df_train_ori['score'].apply(lambda x: randint(0,2) if x == 0 else 1)

In [ ]:
custom_sampled_df = df_train_ori[df_train_ori['select']==1].copy()

In [ ]:
for row in custom_sampled_df["tags"]:
    for tag in row:
        # if tag.startswith("B"):
        #     tag_type = tag.split("-")[1]
        split2freqs['down_sample_1_3'][tag] += 1

pd.DataFrame.from_dict(split2freqs, orient="index")

In [ ]:
tag_counts = pd.DataFrame.from_dict(split2freqs, orient="index")

tag_counts['total_tag'] = tag_counts.sum(axis=1)
tag_counts['entity_ratio'] = tag_counts['O'] / tag_counts['total_tag']
tag_counts

In [ ]:
from math import log,sqrt, ceil

def custom_sampling_based_on_score(df, method):
  # Get stats of the class distribution of the dataset
  labels = sum(df['tags'],[])
  num_tokens = len(labels)
  ent = [label[2:] for label in labels if label != 'O']
  stats = Counter(ent)
  for key in stats:
      #Use frequency instead of count
      stats[key] = stats[key]/num_tokens

  if method not in ['sc','sCR','sCRD','nsCRD']:
    raise ValueError("Unidentified Resampling Method")


  
  # df['score'] = df['tags'].apply(lambda x: sum([ 0 if tag.startswith('O') else 1 for tag in x  ]) + 1)
  
  custom_sampled_df = pd.DataFrame(columns=df.columns)

  # for idx in range(len(df)):
  #     num_samples = df.iloc[idx]['score']
  #     sent = df.iloc[idx].to_dict()

  #     for i in range(num_samples):
  #     # Append the sampled subset to the list
  #       sampled_data.append(sent)
  #       # custom_sampled_df = pd.concat([custom_sampled_df, df.iloc[idx]], ignore_index=True, axis = 0)
  # # Concatenate the sampled subsets to create the final upsampled DataFrame
  # custom_sampled_df = pd.concat([custom_sampled_df, pd.DataFrame(sampled_data)], ignore_index=True)
  
  for sen in range(len(df)):
    sampled_data = [] 
    # Resampling time can at least be 1, which means sentence without 
    # entity will be reserved in the dataset  
    rsp_time = 1
    sen_len = len(df.iloc[sen]['tags'])
    ents = Counter([label[2:] for label in df.iloc[sen]['tags'] if label != 'O'])
          # Pass if there's no entity in a sentence
    
    
    if ents:
      for ent in ents.keys():
        # Resampling method selection and resampling time calculation, 
        # see section 'Resampling Functions' in our paper for details.
        if method == 'sc':
          rsp_time += ents[ent]
        if method == 'sCR' or method == 'sCRD':
          weight = -log(stats[ent],2)
          rsp_time += ents[ent]*weight
        if method == 'nsCRD':
          weight = -log(stats[ent],2)
          rsp_time += sqrt(ents[ent])*weight
        if method == 'sCR':
          rsp_time = sqrt(rsp_time)
        if method == 'sCRD' or method == 'nsCRD':
          rsp_time = rsp_time/sqrt(sen_len)
      # Ceiling to ensure the integrity of resamling time
      rsp_time = ceil(rsp_time) 
    
    for t in range(rsp_time):
      for token in range(sen_len):
        sampled_data.append(df.iloc[sen].to_dict())


    custom_sampled_df = pd.concat([custom_sampled_df, pd.DataFrame(sampled_data)], ignore_index=True)




  return custom_sampled_df

